In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
#|default_exp converter.yolo

# Yolo converter.

> Convert dataset formats.

In [ ]:
#|export
from fastcore.all import *
import shutil
import polvo as pv

/home/lgvaz/git/polvo/polvo/utils/logging.py:8: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [ ]:
#|export
class YOLO(pv.Visitor):
    def convert_dataset(self, save_dir, train_records=None, valid_records=None, test_records=None):
        save_dir = Path(save_dir)
        if train_records: self.convert_records(pv.mkdir(save_dir/'train'), train_records)
        if valid_records: self.convert_records(pv.mkdir(save_dir/'valid'), valid_records)
        if test_records: self.convert_records(pv.mkdir(save_dir/'test'), test_records)
        
    def convert_records(self, save_dir, records):
        for record in pv.pbar(records): self.convert_record(save_dir, record)
        
    def convert_record(self, save_dir, record):
        image_dir = pv.mkdir(Path(save_dir)/'images')
        ann_dir = pv.mkdir(Path(save_dir)/'labels')
        
        self._labels, self._bboxes = [], []
        self.visit_all(record)
        lines = [' '.join(o) for o in pv.safe_zip(self._labels, self._bboxes)]
        pv.save_txt('\n'.join(lines), ann_dir/self._image_file.with_suffix('.txt').name)
        shutil.copy(self._image_file.absolute(), image_dir/self._image_file.name)
    
    def _visit_image_file(self, image_file): 
        self._w,self._h = pv.image_size(image_file)
        self._image_file = image_file
    
    def _visit_bbox_labelled(self, bbox):
        self._labels.append(str(bbox.label.id))
        points = bbox.normalized_points(self._w,self._h).flatten()
        self._bboxes.append(' '.join(map(str, points)))

In [ ]:
import polvo.bbox as pb

In [ ]:
image_file = pv.ImageFile(str(pv.test.SEG_IMAGE))
class_map = {0: 'a', 1: 'b'}
labels = [pv.Label(i, class_map[i]) for i in [0, 1, 1, 0]]
bboxes = [pb.BBox.from_xyxy(20, 20, 64, 280),
          pb.BBox.from_xywh(10, 72, 478, 72),
          pb.BBox.from_relative_xcycwh(.4, .4, .7, .6, 512, 300),
          pb.OBBox.from_clockwise(256,175, 100,140, 80)]

bboxesl = [pb.BBoxLabeled(bbox, label) for bbox, label in pv.safe_zip(bboxes, labels)]

In [ ]:
record = [image_file, *bboxesl]
converter = YOLO()
converter.convert_record(pv.mkdir('yolo', tmp=True, overwrite=True), record)

In [ ]:
converter.convert_dataset(pv.mkdir('tmp/yolo', parents=True, overwrite=True), train_records=[record])

  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
#TODO: Figure out better way of testing. Maybe have a method that returns the converted lines separated from saving.

In [ ]:
#| hide
from nbdev import nbdev_export
nbdev_export()